## Installation & Dependencies

In [1]:
!pip install xai-sdk

   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.5 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.5 MB 2.3 MB/s eta 0:00:02
   ---------------- ----------------------- 1.8/4.5 MB 3.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.5 MB 3.6 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.5 MB 2.7 MB/s eta 0:00:01
   ------------------------- -------------- 2.9/4.5 MB 2.3 MB/s eta 0:00:01
   ------------------------------------- -- 4.2/4.5 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 4.5/4.5 MB 2.8 MB/s eta 0:00:00

  Attempting uninstall: packaging

    Found existing installation: packaging 24.2

    Uninstalling packaging-24.2:

      Successfully uninstalled packaging-24.2

  Attempting uninstall: grpcio

    Found existing installation: grpcio 1.70.0

    Uninstallin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 25.1.1 requires conda-libmamba-solver>=24.11.0, but you have conda-libmamba-solver 24.1.0 which is incompatible.
langchainhub 0.1.21 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain-core 0.3.39 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.39 which is incompatible.
mlflow-skinny 2.17.0 requires packaging<25, but you have packaging 25.0 which is incompatible.
prefect 3.0.2 requires click<8.2,>=8.0, but you have click 8.2.1 which is incompatible.
prefect 3.0.2 requires packaging<24.3,>=21.3, but you have packaging 25.0 which is incompatible.
prefect 3.0.2 requires rich<14.0,>=11.0, but you have rich 14.0.0 which is incompatible.
prefect 3.0

## Imports & Client Configuration

In [1]:
import os

from xai_sdk import Client
from xai_sdk.chat import system, user

# Initialize the client with extended timeout
client = Client(
    api_key=os.getenv("XAI_API_KEY"),
    timeout=3600,  # Extended timeout for reasoning-intensive tasks
)


## Text‑Based Prescription Analysis

In [2]:
# Create a new chat with system prompt tailored to the prescription assistant
chat = client.chat.create(
    model="grok-4",
    messages=[
        system("""
        You are MedGuide AI, a helpful and intelligent assistant that helps users understand their medical prescriptions.
        You explain each medicine's price, availability, and prescribed duration in a clear and concise manner.
        """),
    ],
)

In [3]:
# Append a user prompt simulating a text-only prescription input
chat.append(
    user("""
My prescription says:
- Paracetamol 500mg, take twice a day for 5 days.
- Azithromycin 250mg, once a day for 3 days.

Can you explain the duration and check if these medicines are commonly available?
""")
)

# Get the response from Grok
response = chat.sample()

# Print detailed response information
print("Reasoning Content:")
print(response.reasoning_content)

print("\nFinal Response:")
print(response.content)

print("\nNumber of completion tokens:")
print(response.usage.completion_tokens)

print("Number of reasoning tokens:")
print(response.usage.reasoning_tokens)


Reasoning Content:


Final Response:
Below, I'll explain your prescription based on the details you provided. As MedGuide AI, I'll cover the prescribed duration for each medicine, their common availability, and approximate pricing (note: prices can vary by location, brand, pharmacy, and whether it's generic or branded; these are general estimates based on global averages as of my last update in 2023. Always check with your local pharmacy for current details). Remember, I'm not a substitute for professional medical advice—consult your doctor or pharmacist for personalized guidance, and follow the prescription exactly as directed.

### 1. **Paracetamol 500mg**
   - **Prescribed Duration**: Take one tablet twice a day (e.g., morning and evening) for 5 days. This means a total course of 10 tablets (2 per day × 5 days). It's commonly used for pain relief, fever reduction, or mild inflammation.
   - **Availability**: Very commonly available over-the-counter (no prescription needed in most co

## Image‑Based Prescription Analysis

In [4]:
import base64
from xai_sdk.chat import image

# Set the path to your prescription image
image_path = "data/image1.jpg"

# Start a new chat with Grok 4
chat = client.chat.create(model="grok-4")

# Define the AI's role and behavior
chat.append(
    system(
        "You are MedGuide AI, a helpful and intelligent assistant that reads medical prescriptions and explains each medicine's price, availability, and prescribed duration. Be concise, clear, and accurate."
    )
)


# Function to encode image in base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
        return encoded_string


# Encode the image
base64_image = encode_image(image_path)

# Upload the image with a user query
chat.append(
    user(
        "Please analyze this prescription and tell me the details, price, availability, and duration for each medicine.",
        image(image_url=f"data:image/jpeg;base64,{base64_image}", detail="high"),
    )
)

# Get the AI response
response = chat.sample()
print(response.content)


Below is a concise analysis of the prescription dated 13/10/2022 for Ms. Fiza Mozawala (19/F) from The White Tusk dental clinic. I've interpreted the handwritten items based on common medical terminology and brands (likely from an Indian context). Each entry includes:

- **Details**: What the medicine/item is, its purpose, and dosage.
- **Price**: Approximate retail price in INR (based on current Indian market averages; prices may vary by location/pharmacy; check locally for accuracy).
- **Availability**: Where it's typically available (e.g., pharmacies, online).
- **Prescribed Duration**: As per the prescription.

Note: This appears to be post-dental procedure care (e.g., for infection, pain, and oral hygiene). Consult your doctor/pharmacist for exact usage, side effects, or alternatives. Prices are estimates as of my last update and exclude taxes/discounts.

### 1. Cap. Phexin 500mg
   - **Details**: Cephalexin (antibiotic) capsule for treating bacterial infections (e.g., dental absc

In [5]:
print(response.usage)

completion_tokens: 893
prompt_tokens: 1860
total_tokens: 3351
prompt_text_tokens: 68
prompt_image_tokens: 1792
reasoning_tokens: 598
cached_prompt_text_tokens: 4



## Medicine Info Fetch Utilities

In [ ]:
import os
import json
import base64
from IPython.display import Markdown
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List
from xai_sdk import Client
from xai_sdk.chat import system, user, image, tool, tool_result
from firecrawl import FirecrawlApp, ScrapeOptions


# Initialize clients with connection pooling
client = Client(
    api_key=os.getenv("XAI_API_KEY"),
    timeout=3600,
)
fc = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))


def get_medicine_info_fast(name: str) -> Dict:
    """Optimized medicine info fetcher with error handling"""
    try:
        results = fc.search(
            query=f"{name} medicine price availability",  # Shorter query for faster search
            limit=1,
            scrape_options=ScrapeOptions(formats=["markdown"]),
        )
        snippet = results.data[0] if results.data else {}
        return {
            "name": name,
            "info_markdown": snippet.get("markdown", "N/A"),
            "url": snippet.get("url", "N/A"),
            "description": snippet.get("description", "N/A"),
            "status": "success",
        }
    except Exception as e:
        return {
            "name": name,
            "info_markdown": "Error fetching data",
            "url": "N/A",
            "description": f"Error: {str(e)}",
            "status": "error",
        }


def get_multiple_medicines_concurrent(
    medicine_names: List[str], max_workers: int = 5
) -> List[Dict]:
    """Fetch multiple medicine info concurrently"""
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_medicine = {
            executor.submit(get_medicine_info_fast, name): name
            for name in medicine_names
        }

        # Collect results as they complete
        for future in as_completed(future_to_medicine):
            try:
                result = future.result(timeout=30)  # 30 second timeout per request
                results.append(result)
            except Exception as e:
                medicine_name = future_to_medicine[future]
                results.append(
                    {
                        "name": medicine_name,
                        "info_markdown": "Timeout or error",
                        "url": "N/A",
                        "description": f"Error: {str(e)}",
                        "status": "error",
                    }
                )

    return results


## Tool Definitions & Mapping

In [ ]:
# Tool definitions
tool_definitions = [
    tool(
        name="get_medicine_info_fast",
        description="Fetch markdown info, URL, and description for a medicine via Firecrawl (optimized)",
        parameters={
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Name of the medicine"},
            },
            "required": ["name"],
        },
    ),
    tool(
        name="get_multiple_medicines_concurrent",
        description="Fetch info for multiple medicines concurrently",
        parameters={
            "type": "object",
            "properties": {
                "medicine_names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of medicine names",
                },
                "max_workers": {
                    "type": "integer",
                    "description": "Maximum concurrent workers (default: 5)",
                    "default": 5,
                },
            },
            "required": ["medicine_names"],
        },
    ),
]

# Map tools to functions
tools_map = {
    "get_medicine_info_fast": get_medicine_info_fast,
    "get_multiple_medicines_concurrent": get_multiple_medicines_concurrent,
}

## Enhanced Prescription Analysis with Tools

In [ ]:
def encode_image(path: str) -> str:
    """Utility to encode image to base64"""
    try:
        with open(path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except FileNotFoundError:
        raise FileNotFoundError(f"Image file not found: {path}")
    except Exception as e:
        raise Exception(f"Error encoding image: {str(e)}")


def analyze_prescription_fast(image_path: str) -> str:
    """Main function to analyze prescription with optimizations"""
    try:
        # Encode the prescription image
        encoded_img = encode_image(image_path)

        # Create chat session
        chat = client.chat.create(
            model="grok-4",
            tools=tool_definitions,
            tool_choice="auto",
        )

        # Enhanced system prompt for better extraction
        chat.append(
            system(
                "You are MedGuide AI. Extract ALL medicine names from the prescription image. "
                "If you find multiple medicines, use get_multiple_medicines_concurrent to fetch "
                "all information at once for faster processing. For single medicine, use get_medicine_info_fast. "
                "Create a comprehensive markdown report."
            )
        )

        # User provides the prescription image
        chat.append(
            user(
                "Extract all medicine names from this prescription and get their details efficiently.",
                image(image_url=f"data:image/jpeg;base64,{encoded_img}", detail="high"),
            )
        )

        # Initial model call
        response = chat.sample()
        chat.append(response)

        # Execute tool calls if any
        if response.tool_calls:
            for tc in response.tool_calls:
                func_name = tc.function.name
                func_args = json.loads(tc.function.arguments)

                print(f"Executing {func_name} with args: {func_args}")

                result = tools_map[func_name](**func_args)
                chat.append(tool_result(json.dumps(result)))

            # Request final formatted report
            chat.append(
                user(
                    "Create a comprehensive markdown report with H2 heading for each medicine that contains: Description, "
                    "Typical Duration, Price Information, and Purchase Link"
                )
            )

            # Generate final report
            final = chat.sample()
            return final.content
        else:
            return response.content

    except Exception as e:
        return f"Error analyzing prescription: {str(e)}"


## Demonstration of Fetch Functions

In [7]:
print(get_medicine_info_fast("Aspirin"))

{'name': 'Aspirin', 'info_markdown': '[Skip to main content](https://www.goodrx.com/aspirin#skip-to-content)\n\nAre you a healthcare professional? [Join GoodRx for HCPs](https://www.goodrx.com/hcp/join?redirectTo=%2Faspirin)\n\nSavings Alert\n\n# Aspirin\n\n![Aspirin](https://www.grxstatic.com/d4fuqqd5l3dbz/products/cwf_tms/DrugItem_11031.PNG?width=3840&quality=85&auto=webp)\n\n[Medicare](https://www.goodrx.com/aspirin/medicare-coverage) [Drug Info](https://www.goodrx.com/aspirin/what-is) [Side Effects](https://www.goodrx.com/aspirin/aspirin-side-effects "Aspirin Side Effects") [Dosage](https://www.goodrx.com/aspirin/dosage "Aspirin Dosage")\n\n![Aspirin](https://www.grxstatic.com/d4fuqqd5l3dbz/products/cwf_tms/DrugItem_11031.PNG?width=3840&quality=85&auto=webp)\n\n### Check your prescription details\n\nPrescriptionAspirin 81mg (30 tablets)\n\nChoose pharmacyCheney, KS\n\n- ![Dillons Pharmacy logo](https://www.grxstatic.com/pharmacy_logos/circle_icon/20043.png?width=96&quality=85&auto=

In [8]:
print(get_multiple_medicines_concurrent(["Aspirin", "Ibuprofen"]))

[{'name': 'Aspirin', 'info_markdown': '[Skip to main content](https://www.goodrx.com/aspirin#skip-to-content)\n\nAre you a healthcare professional? [Join GoodRx for HCPs](https://www.goodrx.com/hcp/join?redirectTo=%2Faspirin)\n\nSavings Alert\n\n# Aspirin\n\n![Aspirin](https://www.grxstatic.com/d4fuqqd5l3dbz/products/cwf_tms/DrugItem_11031.PNG?width=3840&quality=85&auto=webp)\n\n[Medicare](https://www.goodrx.com/aspirin/medicare-coverage) [Drug Info](https://www.goodrx.com/aspirin/what-is) [Side Effects](https://www.goodrx.com/aspirin/aspirin-side-effects "Aspirin Side Effects") [Dosage](https://www.goodrx.com/aspirin/dosage "Aspirin Dosage")\n\n![Aspirin](https://www.grxstatic.com/d4fuqqd5l3dbz/products/cwf_tms/DrugItem_11031.PNG?width=3840&quality=85&auto=webp)\n\n### Check your prescription details\n\nPrescriptionAspirin 81mg (30 tablets)\n\nChoose pharmacyCheney, KS\n\n- ![Dillons Pharmacy logo](https://www.grxstatic.com/pharmacy_logos/circle_icon/20043.png?width=96&quality=85&auto

## Run the Optimized End‑to‑End Analysis

In [ ]:
# Run the optimized analysis
image_path = "data/image1.jpg"
result = analyze_prescription_fast(image_path)


Markdown(result)


Executing get_multiple_medicines_concurrent with args: {'medicine_names': ['Phexin 500mg', 'Zerodol PT', 'Stolin gum paint', 'Colgate Plax mouth wash']}


# Comprehensive Medicine Report

This report provides details on the medicines extracted from the prescription: Phexin 500mg, Zerodol PT, Stolin Gum Paint, and Colgate Plax Mouth Wash. For each medicine, the following information is included: Description (based on available data), Typical Duration (inferred from standard usage guidelines; consult a doctor for personalized advice), Price Information (approximate based on scraped data), and Purchase Link (direct URL from sources like 1mg, Amazon, or eBay). Note: Prices may vary by location, seller, and time. Always consult a healthcare professional before use, as these are prescription or over-the-counter items.

## Phexin 500mg

### Description
Phexin 500mg Capsule is an antibiotic (Cefalexin 500mg) used to treat various bacterial infections, including those of the respiratory tract, ear, skin, bones, and urinary tract. It works by stopping bacteria from building their protective cell walls, leading to bacterial death. Common side effects include diarrhea, nausea, vomiting, indigestion, and abdominal pain.

### Typical Duration
Typically taken for 7-14 days or as prescribed by a doctor, depending on the infection. Complete the full course to prevent resistance, even if symptoms improve.

### Price Information
₹238.5 for 1 strip of 10 capsules (17% off from ₹287.6). Available in India; prices may vary.

### Purchase Link
[Buy on 1mg.com](https://www.1mg.com/drugs/phexin-500mg-capsule-50484)

## Zerodol PT

### Description
Zerodol PT Tablet is a pain-relieving combination medicine (Aceclofenac 100mg + Paracetamol 325mg + Tramadol 37.5mg) used to reduce pain and inflammation in conditions like rheumatoid arthritis, ankylosing spondylitis, osteoarthritis, muscle pain, back pain, toothache, or pain in the ear and throat. It blocks chemical messengers causing pain and reduces fever. Common side effects include nausea, vomiting, stomach pain, indigestion, loss of appetite, dryness in mouth, dizziness, sleepiness, and heartburn.

### Typical Duration
Typically taken for short-term relief (e.g., 5-10 days) or as advised by a doctor. Do not exceed the prescribed duration due to potential habit-forming effects.

### Price Information
₹16.5 per tablet (strip of 10 tablets). Not available for direct sale on some platforms; requires prescription.

### Purchase Link
[View on 1mg.com](https://www.1mg.com/drugs/zerodol-pt-tablet-160347) (Note: Marked as "NOT FOR SALE" on the site; consult a pharmacy or doctor)

## Stolin Gum Paint

### Description
Stolin Gum Paint is an astringent used for treating bleeding, swollen, spongy, and receding/loose gums. It helps prevent mouth ulcers and is applied directly to the gums. Available in packs like Dr. Reddy's Stolin Gum Astringent (15 ml each). Shake well before use. Common uses include gum health and oral care.

### Typical Duration
Applied 1-0-1 (morning and night) for 3-5 days or as directed. Not for long-term use without dental advice.

### Price Information
US $23.74 for a pack of 5 (15 ml each), with bulk savings available (e.g., $23.50 each for 2+). Prices from eBay; international shipping may apply.

### Purchase Link
[Buy on eBay](https://www.ebay.com/itm/266704508636)

## Colgate Plax Mouth Wash

### Description
Colgate Plax Complete Care Mouthwash is an alcohol-free mouthwash (Peppermint flavor, 250ml/8.5oz) that provides fresh breath, kills germs, and helps with bad breath treatment. It offers complete oral care without burning sensation. Common side effects are rare but may include mild stomach upset if swallowed.

### Typical Duration
Used daily (e.g., rinse for 30 seconds, twice a day) for ongoing oral hygiene. One bottle lasts about 1-2 weeks with regular use.

### Price Information
$10.76 for 250ml (27% off from $14.69). Available on Amazon; additional shipping may apply.

### Purchase Link
[Buy on Amazon](https://www.amazon.com/Colgate-Plax-Complete-Care-Mouthwash/dp/B00I6F7612)